# Data cleaning

In [ ]:
#@title imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

from sklearn.metrics import (
    mean_squared_error, mean_absolute_error, r2_score,
    accuracy_score, precision_score, recall_score,
    f1_score, confusion_matrix
)

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
from collections import deque

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import SimpleRNN, Dense, Activation, Dropout, LSTM, GRU, Bidirectional
from tensorflow.keras.optimizers import Adam, Nadam, RMSprop
from tensorflow.keras.utils import pad_sequences, to_categorical
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
def read(file):
    df = pd.read_csv(file, low_memory=False)
    df.columns = df.columns.str.strip()
    return df


def isp(df):
  df["NL_DT"]  = pd.to_datetime(df["NL_DT"], format="%d/%m/%y %H:%M")

  if "ISP" not in df.columns:
    df["ISP"] = df.groupby(df["NL_DT"].dt.date).cumcount() + 1
  df["jaar"] = df["NL_DT"].dt.year
  return df


def drop(df):
  df.drop(columns=['NL_DT'], inplace=True)

  if 'kWh_APX_zon' in df.columns:
    df.drop(columns=['kWh_APX_zon'], inplace=True)
  return df


def obj_float(df):
  for col in df.columns:
    df[col] = df[col].astype(str).str.replace('(', '').str.replace(')', '')
    df[col] = df[col].astype(str).str.replace('-', '0')
    df[col] = df[col].astype(str).str.replace(',', '.', regex=False)
    df[col] = pd.to_numeric(df[col], errors='coerce')
  # NaNs filled with zeros
  df.fillna(0, inplace=True)

  df.sort_values(by=['jaar', 'maand', 'DagM', 'ISP'], inplace=True)
  df.reset_index(drop=True, inplace=True)
  return df


def mwh_to_kwh(df, column):
  df[column] = df[column] / 1000
  return df


def plot_loss(history, title='Loss per Epoch'):
    plt.figure(figsize=(8, 4))
    plt.plot(history.history['loss'], label='Training loss')
    if 'val_loss' in history.history:
        plt.plot(history.history['val_loss'], label='Validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title(title)
    plt.legend()
    plt.grid()
    plt.tight_layout()
    plt.show()


def evaluate_model_results(model, model_name, X_train, X_test, y_train_idx, y_test_idx, duration, future, dataset):
    df = pd.DataFrame(columns=['imbalance_price', 'predicted', 'test'])
    df['imbalance_price'] = [row['Imbalance'] for row in dataset[duration+future:]]

    # predictions
    df.loc[y_train_idx, 'predicted'] = model.predict(X_train).flatten()
    df.loc[y_test_idx, 'predicted'] = model.predict(X_test).flatten()
    df.loc[y_train_idx, 'test'] = False
    df.loc[y_test_idx, 'test'] = True

    # for test/train results
    df_test = df[df['test'] == True]
    df_train = df[df['test'] == False]

    y_true = df_test['imbalance_price'].values
    y_pred = df_test['predicted'].values
    y_train = df_train['imbalance_price'].values
    y_pred_train = df_train['predicted'].values

    # metrics
    r2 = r2_score(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mse_train = mean_squared_error(y_train, y_pred_train)

    mape = np.mean(np.abs((y_true[y_true != 0] - y_pred[y_true != 0]) / y_true[y_true != 0])) * 100
    nrmse = rmse / np.mean(y_true)
    nmae = mae / np.mean(y_true)

    # results
    print(f"\n Results for {model_name}")
    print(f"------------------------")
    print(f"R² Score:       {r2:.4f}")
    print(f"MSE (test):     {mse:.4f}")
    print(f"MSE (train):    {mse_train:.4f}")
    print(f"MAE:            {mae:.4f}")
    print(f"RMSE:           {rmse:.4f}")
    print(f"MAPE:           {mape:.2f}%")
    print(f"nRMSE (mean):   {nrmse:.4f}")
    print(f"nMAE (mean):    {nmae:.4f}")

    # to see if there are peaks in the testdata:
    threshold = 400
    df_peaks = df_test.copy()

    df_peaks["correct_pred"] = df_peaks["imbalance_price"] >= threshold & df_peaks["predicted"] >= threshold

    total = df_peaks["true_peak"].sum()
    correct = df_peaks["correct_pred"].sum()

    print("\n Peak prediction:")
    print(f"Total amount of peaks ≥ {threshold:.0f}: {total}")
    print(f"Correct predicted: {correct}")


def evaluate_classification(model, model_name, X_train, X_test, y_train_idx, y_test_idx, duration, future, dataset, threshold=400):
    df = pd.DataFrame(columns=['imbalance_price', 'predicted', 'test'])
    df['imbalance_price'] = [row['Imbalance'] for row in dataset[duration+future:]]

    # predictions
    df.loc[y_train_idx, 'predicted'] = model.predict(X_train).flatten()
    df.loc[y_test_idx, 'predicted'] = model.predict(X_test).flatten()
    df.loc[y_train_idx, 'test'] = False
    df.loc[y_test_idx, 'test'] = True

    # only testset
    df_test = df[df['test'] == True]

    y_true = df_test['imbalance_price'].values
    y_pred = df_test['predicted'].values

    # convert to binary classes
    y_true_peak = np.array([1 if abs(val) >= threshold else 0 for val in y_true])
    y_pred_peak = np.array([1 if abs(val) >= threshold else 0 for val in y_pred])
    print("y_true_peak:", y_true_peak[:5])
    print("y_pred_peak:", y_pred_peak[:5])


    # metrics
    accuracy = accuracy_score(y_true_peak, y_pred_peak)
    precision = precision_score(y_true_peak, y_pred_peak)
    recall = recall_score(y_true_peak, y_pred_peak)
    f1 = f1_score(y_true_peak, y_pred_peak)
    cm = confusion_matrix(y_true_peak, y_pred_peak)

    # results
    print(f"\n Results for classification: {model_name}")
    print("----------------------------------------")
    print(f"Accuracy:       {accuracy:.4f}")
    print(f"Precision:      {precision:.4f}")
    print(f"Recall:         {recall:.4f}")
    print(f"F1 Score:       {f1:.4f}")
    print("Confusion Matrix:")
    print(cm)


def onbalans_data(file):
  df = pd.read_csv(file, delimiter=';')
  df.columns = df.columns.str.strip()

  df['Timeinterval Start Loc'] = pd.to_datetime(df['Timeinterval Start Loc'])
  df['maand'] = df['Timeinterval Start Loc'].dt.month
  df['DagM'] = df['Timeinterval Start Loc'].dt.day
  df['jaar'] = df['Timeinterval Start Loc'].dt.year

  df.drop(columns=['Timeinterval Start Loc', 'Timeinterval End Loc', 'Quantity Measurement Unit'], inplace=True)
  df.rename(columns={'Isp': 'ISP'}, inplace=True)

  mwh_to_kwh(df, 'Surplus')
  mwh_to_kwh(df, 'Shortage')
  mwh_to_kwh(df, 'Absolute')
  mwh_to_kwh(df, 'Imbalance')

  # column indicating whether there is a peak in the Imbalance
  threshold = 400
  df['piek'] = df['Imbalance'].apply(lambda x: 1 if abs(x) >= threshold else 0)

  return df

# --- Load files --- #
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
file_path = '/content/drive/MyDrive/Thesis_VON'

df_jan2021 = read(f'{file_path}/2021_jan-jun.csv')
df_jul2021 = read(f'{file_path}/2021_jul-dec.csv')
df_2022 = read(f'{file_path}/2022.csv')
df_2023 = read(f'{file_path}/2023.csv')
df_2024 = read(f'{file_path}/zonneproductie_2024.csv')


# change the column names so they are the same (2021)
df_jan2021.rename(columns={'kWh_real': 'kWh_real_zon', 'kWh_nom': 'kWh_nom_zon',
                           'kWh_APX': 'kWh_APX_zon', 'kWh_shor': 'kWh_shor_zon',
                           'kWh_sur': 'kWh_sur_zon', 'eur_APX': 'eur_APX_zon',
                           'eur_sur': 'eur_sur_zon', 'eur_shor': 'eur_shor_zon',
                           'eur_tot': 'eur_tot_zon'}, inplace=True)
df_2023.rename(columns={'eur_short_zon': 'eur_shor_zon', 'p_short': 'p_shor',
                        'eur_real_zon': 'eur_tot_zon'}, inplace=True)
df_2024.rename(columns={'eur_short_zon': 'eur_shor_zon', 'p_short': 'p_shor',
                        'eur_real_zon': 'eur_tot_zon'}, inplace=True)
# remove the unnecessary columns
df_jan2021.drop(columns=['kWh_Tele', 'kWh_prof', 'kWh_forward', 'kWh_imb',
                         'EUR prof real', 'p_forward', 'eur_forward', 'NL_DT.1',
                         'kWh_levering', 'kWh_terug', 'aantal_eans', 'kWh_APX_inkoop',
                         'kWh_APX_verkoop', 'NL_DT.2', 'kWh_nom_zonnepark'], inplace=True)
df_2022.drop(columns=['tijd'], inplace=True)
df_2024.drop(columns=['Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21'], inplace=True)

# make one dataset for 2021
df_2021 = pd.concat([df_jan2021, df_jul2021])

# add ISP + drop
df_2021 = obj_float(drop(isp(df_2021)))
df_2022 = obj_float(drop(isp(df_2022)))
df_2023 = obj_float(drop(isp(df_2023)))

# Special case: df_2024 (no NL_DT)
df_2024 = drop(df_2024)
df_2024["jaar"] = 2024
df_2024 = obj_float(df_2024)

df_all = pd.concat([df_2021, df_2022, df_2023, df_2024], ignore_index=True)

df_onbalans2021 = onbalans_data(f'{file_path}/settled_imbalance_2021.csv')
df_onbalans2022 = onbalans_data(f'{file_path}/settled_imbalance_2022.csv')
df_onbalans2023 = onbalans_data(f'{file_path}/settled_imbalance_2023.csv')
df_onbalans2024 = onbalans_data(f'{file_path}/settled_imbalance_2024.csv')

df_onbalans = pd.concat([df_onbalans2021, df_onbalans2022, df_onbalans2023, df_onbalans2024], ignore_index=True)

df_samen = pd.merge(df_all, df_onbalans, on=['maand', 'DagM', 'ISP', 'jaar'], how='left')
# due to leap year in 2024 there are nans in it
df_samen.dropna(inplace=True)

# save to csv for in the drive
df_samen.to_csv(f'{file_path}/all_data.csv', index=False)

Mounted at /content/drive
Mounted at /content/drive


In [ ]:
#@title Models
def build_vanilla(neurons, lr):
    """
    Builds a Vanilla RNN model with arbitrary depth.

    Parameters:
    - neurons: in a list, one per RNN layer (e.g., [32], [64, 32], etc.)
    - lr: learning rate
    """
    model = Sequential()

    for i, n in enumerate(neurons):
        # return_sequences for all layer, instead of last layer
        return_seq = i < len(neurons) - 1
        model.add(SimpleRNN(n, activation='tanh', return_sequences=return_seq))

    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer=RMSprop(learning_rate=lr))
    return model

def build_lstm(neurons, lr):
    model = Sequential()

    for i, n in enumerate(neurons):
        # return_sequences for all layer, instead of last layer
        return_seq = i < len(neurons) - 1
        model.add(LSTM(n, activation='tanh', return_sequences=return_seq))

    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=lr))
    return model

def build_gru(neurons, lr):
    model = Sequential()
    for i, n in enumerate(neurons):
        # return_sequences for all layer, instead of last layer
        return_seq = i < len(neurons) - 1
        model.add(GRU(n, activation='tanh', return_sequences=return_seq))

    model.add(Dense(1))
    adam = optimizers.Adam(learning_rate=0.0001)
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def build_biRNN(neurons, lr):
    model = Sequential()
    model = Sequential()
    for i, n in enumerate(neurons):
        # return_sequences for all layer, instead of last layer
        return_seq = i < len(neurons) - 1
        model.add(Bidirectional(SimpleRNN(n, activation='tanh', return_sequences=return_seq)))

    model.add(Dense(1))
    adam = optimizers.Adam(learning_rate=0.0001)
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model